In [316]:
import pandas as pd
import numpy as np
companies_df = pd.read_csv('companies.csv')
investments_df = pd.read_csv('investments.csv')
import networkx as nx

In [287]:
companies_df = companies_df.loc[companies_df['country_code'] == "HKG"]
investments_df = investments_df.loc[investments_df['company_country_code'] == "HKG"]

In [288]:
companies_df.columns

Index(['permalink', 'name', 'homepage_url', 'category_list',
       'funding_total_usd', 'status', 'country_code', 'state_code', 'region',
       'city', 'funding_rounds', 'founded_at', 'first_funding_at',
       'last_funding_at'],
      dtype='object')

In [289]:
investments_df.columns

Index(['company_permalink', 'company_name', 'company_category_list',
       'company_country_code', 'company_state_code', 'company_region',
       'company_city', 'investor_permalink', 'investor_name',
       'investor_country_code', 'investor_state_code', 'investor_region',
       'investor_city', 'funding_round_permalink', 'funding_round_type',
       'funding_round_code', 'funded_at', 'raised_amount_usd'],
      dtype='object')

In [290]:
entities = {}

def add_country(country_code, entities, edge_to):
    if not pd.isnull(country_code):
        if country_code not in entities:
            entities[country_code] = {
                'name': country_code,
                'type': 'country',
                'edges': []
            }
        entities[country_code]['edges'].append(edge_to)
    

for index, row in companies_df.iterrows():
    row['permalink'] = row['permalink'].split('/')[2]
    if row['permalink'] not in entities:
        entities[row['permalink']] = {}
        entities[row['permalink']]['type'] = 'company'
        entities[row['permalink']]['country'] = row['country_code']
        add_country(row['country_code'], entities, row['permalink'])
        
        entities[row['permalink']]['name'] = row['name']
        entities[row['permalink']]['category'] = row['category_list']
        entities[row['permalink']]['funding_total_usd'] = row['funding_total_usd']
        entities[row['permalink']]['investments'] = []
        entities[row['permalink']]['edges'] = []

In [291]:
for index, row in investments_df.iterrows():
    row['investor_permalink'] = row['investor_permalink'].split('/')[2]
    row["company_permalink"] = row["company_permalink"].split('/')[2]
    if row['investor_permalink'] not in entities:
        entities[row['investor_permalink']] = {}
        entities[row['investor_permalink']]['name'] = row['investor_name']
        entities[row['investor_permalink']]['type'] = 'investor'
        entities[row['investor_permalink']]['country'] = row['investor_country_code']
        add_country(row['investor_country_code'], entities, row['investor_permalink'])
        entities[row['investor_permalink']]['investments'] = []
        entities[row['investor_permalink']]['edges'] = []
    
    entities[row['investor_permalink']]['investments'].append({
        "company_permalink":row["company_permalink"],
        "company_name":row["company_name"],
        "company_country_code":row["company_country_code"],
        "funding_total_usd":row["raised_amount_usd"],
        "funding_round_type":row["funding_round_type"]
    })
    
    entities[row['investor_permalink']]['edges'].append(row["company_permalink"])
    
    if row["company_permalink"] not in entities:
        entities[row['company_permalink']] = {}
        entities[row['company_permalink']]['type'] = 'company'
        entities[row['company_permalink']]['country'] = row['company_country_code']
        add_country(row['company_country_code'], entities, row['company_permalink'])
        entities[row['company_permalink']]['name'] = row['company_name']
        entities[row['company_permalink']]['category'] = row['company_category_list']
        entities[row['company_permalink']]['investments'] = []
        entities[row['company_permalink']]['edges'] = []

In [294]:
news_sources = requests.get('https://newsapi.org/v2/sources?apiKey=bdf4e4feaf0647f4bb55ef2293f55e82').json()['sources']

In [295]:
for source in news_sources:
    ns[source['id']] = source

In [296]:
import requests
import pycountry
import copy
print(len(entities))
old_entities = copy.deepcopy(entities)
                    

427


In [300]:
import urllib.parse
count = 1
for key, entity in old_entities.items():
    if entity['type'] != 'country':
        print(str(count) + ' ' + entity['name'])
        count += 1
        if count >= 201:
            f = { 'q' : '"'+entity['name']+'"'}
            results = requests.get('https://newsapi.org/v2/everything?apiKey=f3c61387d5e24cdc88adaabd1cb12379&'+urllib.parse.urlencode(f)).json()
            for result in results['articles']:
                source_id = result['source']['id']
                if source_id in ns.keys():
                    if source_id not in entities:
                        entities[source_id] = {
                            'name':ns[source_id]['name'],
                            'type':'news',
                            'edges':[],
                            'mentions':[],
                        }
                        if ns[source_id]['country'] == "zh":
                            country_code = "CHN"
                        else:
                            country_code = pycountry.countries.get(alpha_2=ns[source_id]['country'].upper()).alpha_3
                        add_country(country_code, entities, source_id)
                    entities[source_id]['edges'].append(key)
                    entities[source_id]['mentions'].append({
                        'entity_id':key,
                        'result':result
                    })

1 01Games Technology
2 8 Securities
3 9GAG
4 Abacuz Limited
5 AcesoBee
6 AfterShip
7 AgileSource
8 alife studios inc
9 alive.cn
10 Allied Pacific Sports Network
11 Altai Technologies
12 American Life Media
13 AMP Credit Technologies
14 Amperex Technology
15 Animoca Brands Corporation
16 Anxa
17 Snapask
18 Appeon Corporation
19 Appy Hotel
20 apta.me
21 Ascentage Pharma
22 Aviasales
23 Avrio Solutions Company Limited
24 Bespokify
25 Big Dipper Studio
26 Bioserie
27 BitMEX - Bitcoin Mercantile Exchange
28 Bitspark
29 Boxful
30 BUTLUR
31 Cashange
32 CashYou
33 CatchMe!
34 CDC Corporation
35 China-8
36 China Everbright International
37 Alibaba Pictures Group Limited
38 CinemaWell.com
39 Cino Internetwork
40 CMGE
41 CoachBase
42 Codex Genetics
43 CompareAsiaGroup
44 Crowdbaron
45 Cryptonator
46 Dataflow
47 Daydaycook
48 Delivery Republic
49 DigiSat Technology
50 Dinero Limited
51 Dinglepharb
52 DIY.watch
53 Dragon Law
54 E-Fever
55 Easyship Fulfillment Services
56 EasyVan by lalamove
57 EcoP

In [337]:
for key, entity in entities.items():
    if entity["type"] == "news":
        for i in range(len(entity['edges'])):
            for o_key, o_entity in old_entities.items():
                if o_entity == entity['edges'][i]:
                    entity['edges'][i] = o_key
                    entity['mentions'][i]['entity_id'] = o_key
                    break

In [338]:
import json
with open('entities.json', 'w') as outfile:
    json.dump(entities, outfile)

In [309]:
def get_group(type):
    if type == "company":
        return 1
    if type == "investor":
        return 2
    if type == "news":
        return 3
    if type == "country":
        return 4
    return 5
graph = {
    "nodes":[],
    "links":[]
}
for key, entity in entities.items():
    graph["nodes"].append({
        "id":key,
        "group":get_group(entity['type'])
    })
    for edge_to in entity['edges']:
        graph["links"].append({
        "source":key,
        "target":edge_to,
        "value":1
    })

In [339]:
with open('crunchbase.json', 'w') as outfile:
    json.dump(graph, outfile)

In [319]:
!pip install networkx
import networkx as nx
G = nx.Graph()

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [322]:
G = nx.Graph()
for key,entity in entities.items():
    for edge in entity['edges']:
        G.add_edge(key,edge)

01games-technology
HKG
8-securities-limited
9gag
abacuz-limited
acesobee
aftership
agilesource
alife-studios-inc
alive-cn
allied-pacific-sports-network
altai-technologies
american-life-media
amp-credit-technologies
amperex-technology
animoca-brands-corporation
anxa
appedu
appeon-corporation
appyhotel
apta-me
ascentage-pharma
aviasales-ru
avrio-solutions-company-limited
bespokify
big-dipper-studio
bioserie
bitmex---bitcoin-mercantile-exchange
bitspark
boxful
butlur
cashange
cashyou
catchme
cdc-corp
china
china-everbright-international
chinavision
cinemawell-com
cino-internetwork
cmge
coachbase
codex-genetics
compare-asia-group
crowdbaron
cryptonator
dataflow
daydaycook
delivery-republic
digisat-technology
dinero-limited
dinglepharb
diy-watch
dragon-law
e-fever
easyship-fulfillment-services
easyvan
ecoport
eddingpharm-cayman
editgrid
english-tv
epicforce
exicon
ezeecube
falafel-games
floship
fly-taxi
flyingcodes
frenzoo
friends-of-the-family-hk-social-enterprise
g-innovator-research-crea

In [335]:
paths = nx.all_simple_paths(G, source='aftership', target='easyvan',cutoff=8)

In [336]:
print(list(paths))

[['aftership', 'HKG', '01games-technology', 'cyberport-hong-kong', 'english-tv', 'google-news', 'mindworks-ventures', 'easyvan'], ['aftership', 'HKG', '01games-technology', 'cyberport-hong-kong', 'ezeecube', 'techcrunch', 'mindworks-ventures', 'easyvan'], ['aftership', 'HKG', '01games-technology', 'cyberport-hong-kong', 'kites', 'google-news', 'mindworks-ventures', 'easyvan'], ['aftership', 'HKG', '01games-technology', 'cyberport-hong-kong', 'klook', 'matrix-partners', 'google-news', 'mindworks-ventures', 'easyvan'], ['aftership', 'HKG', '01games-technology', 'cyberport-hong-kong', 'shopline', 'ardent-capital', 'techcrunch', 'mindworks-ventures', 'easyvan'], ['aftership', 'HKG', '01games-technology', 'cyberport-hong-kong', 'shopline', 'coent-venture-partners', 'techcrunch', 'mindworks-ventures', 'easyvan'], ['aftership', 'HKG', '01games-technology', 'cyberport-hong-kong', 'smart-devices', 'google-news', 'mindworks-ventures', 'easyvan'], ['aftership', 'HKG', '01games-technology', 'cyber